![alt text](https://i.imgur.com/1WaY7aA.png)

## Lab 4.2.2: Feature Selection

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

%matplotlib inline

### 1. Forward Feature Selection

> Forward Selection: Forward selection is an iterative method in which we start with having no feature in the model. In each iteration, we keep adding the feature which best improves our model till an addition of a new variable does not improve the performance of the model.

Create a Regression model using Forward Feature Selection by looping over all the features adding one at a time until there are no improvements on the prediction metric ( R2  and  AdjustedR2  in this case).

#### 1.1 Load Wine Data & Define Predictor and Target

In [2]:
## Load the wine quality dataset
wine_csv = '../DATA/winequality_merged.csv'

# Load the wine dataset from csv
wine = pd.read_csv(wine_csv)


# define the target variable (dependent variable) as y
y = wine['quality']

# Take all columns except target as predictor columns
predictor_columns = [c for c in wine.columns if c != 'quality']
# Load the dataset as a pandas data frame
X = pd.DataFrame(wine, columns = predictor_columns)

In [3]:
wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red_wine
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


In [4]:
## Create training and testing subsets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

#### 1.2 Overview of the code below

The external `while` loop goes forever until there are no improvements to the model, which is controlled by the flag `changed` (until is **not** changed).
The inner `for` loop goes over each of the features not yet included in the model and calculates the correlation coefficient. If any model improves on the previous best model then the records are updated.

#### Code variables
- `included`: list of the features (predictors) that were included in the model; starts empty.
- `excluded`: list of features that have **not** been included in the model; starts as the full list of features.
- `best`: dictionary to keep record of the best model found at any stage; starts 'empty'.
- `model`: object of class LinearRegression, with default values for all parameters.

#### Methods of the `LinearRegression` object to investigate
- `fit()`
- `fit.score()`

#### Adjusted $R^2$ formula
$$Adjusted \; R^2 = 1 - { (1 - R^2) (n - 1)  \over n - k - 1 }$$

#### Linear Regression [reference](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)

In [5]:
def adjusted_R(r_squared,n,k):
    return 1 - ((1-r_squared)*(n-1))/(n-k-1)
    

In [6]:
## Flag intermediate output

show_steps = True   # for testing/debugging
# show_steps = False  # without showing steps

In [7]:
## Use Forward Feature Selection to pick a good model

# start with no predictors
included = []
# keep track of model and parameters
best = {'feature': '', 'r2': 0, 'a_r2': 0}
# create a model object to hold the modelling parameters
model = LinearRegression() # create a model for Linear Regression
# get the number of cases in the test data
n = X_test.shape[0]

while True:
    changed = False
    
    if show_steps:
        print('') 

    # list the features to be evaluated
    excluded = list(set(X.columns) - set(included))
    
    if show_steps:
        print('(Step) Excluded = %s' % ', '.join(excluded))  

    # for each remaining feature to be evaluated
    for new_column in excluded:
        
        if show_steps:
            print('(Step) Trying %s...' % new_column)
            print('(Step) - Features = %s' % ', '.join(included + [new_column]))

        # fit the model with the Training data
        fit = model.fit(X_train[included + [new_column]], y_train) # fit a model; consider which predictors should be included
        # calculate the score (R^2 for Regression)
        r2 = model.score(X_test[included + [new_column]], y_test) # calculate the score
        # number of predictors in this model
        k = len(included + [new_column]) + 1
        # calculate the adjusted R^2
        adjusted_r2 = adjusted_R(r2,n,k) 

        if show_steps:
            print('(Step) - Adjusted R^2: This = %.3f; Best = %.3f' % 
                  (adjusted_r2, best['a_r2']))

        # if model improves
        if adjusted_r2 > best['a_r2']:
            # record new parameters
            best = {'feature': new_column, 'r2': r2, 'a_r2': adjusted_r2}
            # flag that found a better model
            changed = True
            if show_steps:
                print('(Step) - New Best!   : Feature = %s; R^2 = %.3f; Adjusted R^2 = %.3f' % 
                      (best['feature'], best['r2'], best['a_r2']))
    # END for

    # if found a better model after testing all remaining features
    if changed:
        # update control details
        included.append(best['feature'])
        excluded = list(set(excluded) - set(best['feature']))
        print('Added feature %-4s with R^2 = %.3f and adjusted R^2 = %.3f' % 
              (best['feature'], best['r2'], best['a_r2']))
    else:
        # terminate if no better model
        break

print('')
print('Resulting features:')
print(', '.join(included))


(Step) Excluded = citric acid, density, residual sugar, free sulfur dioxide, volatile acidity, pH, total sulfur dioxide, chlorides, red_wine, alcohol, sulphates, fixed acidity
(Step) Trying citric acid...
(Step) - Features = citric acid
(Step) - Adjusted R^2: This = 0.007; Best = 0.000
(Step) - New Best!   : Feature = citric acid; R^2 = 0.009; Adjusted R^2 = 0.007
(Step) Trying density...
(Step) - Features = density
(Step) - Adjusted R^2: This = 0.083; Best = 0.007
(Step) - New Best!   : Feature = density; R^2 = 0.084; Adjusted R^2 = 0.083
(Step) Trying residual sugar...
(Step) - Features = residual sugar
(Step) - Adjusted R^2: This = -0.002; Best = 0.083
(Step) Trying free sulfur dioxide...
(Step) - Features = free sulfur dioxide
(Step) - Adjusted R^2: This = -0.003; Best = 0.083
(Step) Trying volatile acidity...
(Step) - Features = volatile acidity
(Step) - Adjusted R^2: This = 0.065; Best = 0.083
(Step) Trying pH...
(Step) - Features = pH
(Step) - Adjusted R^2: This = -0.003; Best 

(Step) - Adjusted R^2: This = 0.280; Best = 0.278
(Step) - New Best!   : Feature = red_wine; R^2 = 0.283; Adjusted R^2 = 0.280
(Step) Trying fixed acidity...
(Step) - Features = alcohol, volatile acidity, sulphates, residual sugar, fixed acidity
(Step) - Adjusted R^2: This = 0.272; Best = 0.280
Added feature red_wine with R^2 = 0.283 and adjusted R^2 = 0.280

(Step) Excluded = citric acid, density, free sulfur dioxide, pH, total sulfur dioxide, chlorides, fixed acidity
(Step) Trying citric acid...
(Step) - Features = alcohol, volatile acidity, sulphates, residual sugar, red_wine, citric acid
(Step) - Adjusted R^2: This = 0.280; Best = 0.280
(Step) - New Best!   : Feature = citric acid; R^2 = 0.284; Adjusted R^2 = 0.280
(Step) Trying density...
(Step) - Features = alcohol, volatile acidity, sulphates, residual sugar, red_wine, density
(Step) - Adjusted R^2: This = 0.283; Best = 0.280
(Step) - New Best!   : Feature = density; R^2 = 0.286; Adjusted R^2 = 0.283
(Step) Trying free sulfur di